In [1]:
from IPython.core.display import display, HTML

### Widen notebook to fit browser window
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os, glob
import pickle
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np

from configuration import dict_config_params
from sliding_windows import search_windows, slide_window, draw_boxes, find_cars
from sliding_windows import add_heat, apply_heat_threshold, draw_labeled_bboxes, get_heat_based_bboxes
from classifiers import get_training_data, fit_svm

<IPython.core.display.Javascript object>

In [3]:
def main_test_images():    
    
    # Test images    
    count_fig = 1
    filenames_img = glob.glob('./test_images/test*.jpg')
    for filename_img in filenames_img:
        
        # Read image
        image = mpimg.imread(filename_img)

        # Normalize image intensity, since we read in a jpg
        image = image.astype(np.float32)/255
        
        y_start_stop = [np.int(image.shape[0]/2), image.shape[0] - 60]        
        hot_windows = []
        if True:
            # Search for cars directly with HOG subsampling
            for scale in np.arange(1, 2, 0.5):
                window_img, windows = find_cars(image, y_start_stop, svc, X_scaler, scale=scale)
                hot_windows += windows
        else:            
            # No HOG subsampling (HOG features extracted on every search window separately)
            windows = slide_window(image, x_start_stop=[None, None], y_start_stop=y_start_stop, 
                            xy_window=(96, 96), xy_overlap=(0.5, 0.5))

            hot_windows = search_windows(image, windows, svc, X_scaler)                       
            window_img = draw_boxes(draw_image, hot_windows, color=(0, 0, 255), thick=6)                

        draw_img, heatmap = get_heat_based_bboxes(image, hot_windows)
            
        fig = plt.figure(count_fig)
        plt.subplot(121)
        plt.imshow(draw_img)
        plt.title('Car Positions')
        plt.subplot(122)
        plt.imshow(heatmap, cmap='hot')
        plt.title('Heat Map')
        fig.tight_layout()
        count_fig += 1
    
    plt.show()

In [4]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

def process_video_frame(frame):
    # NOTE: output should be a color image (3 channel) for processing video below    
    global dict_config_params
    global num_frames_processed  
    global svc, X_scaler    
        
    # Normalize image intensity (?)
    #frame = frame.astype(np.float32)/255
    
    y_start_stop = [np.int(frame.shape[0]/2), frame.shape[0] - 60]    
    hot_windows = []    
    
    # Search for cars directly with HOG subsampling
    for scale in np.arange(1, 2, 0.5):
        window_img, windows = find_cars(frame, y_start_stop, svc, X_scaler, scale=scale)
        hot_windows += windows
        
    draw_frame, heatmap = get_heat_based_bboxes(frame, hot_windows)
    return draw_frame


def main_video():
    clip1 = VideoFileClip("./videos/test_video.mp4")
    #clip1 = VideoFileClip("./videos/project_video.mp4")
    
    clip = clip1.fl_image(process_video_frame)
    clip.write_videofile("out_video.mp4", audio=False)
    

In [5]:
# Globals
svc, X_scaler = None, None

if __name__ == '__main__':
    
    # Pickle dump classifier
    filename_classifier = "./classifiers_pickle.p"
    if (dict_config_params['use_cached_classifier'] is True and
            os.path.exists(filename_classifier)):
                print("\nUsing already available cached classifier.")
                svc, X_scaler = pickle.load( open(filename_classifier, "rb" ) )
    else:
        X, labels = get_training_data()    
        svc, X_scaler = fit_svm(X, labels, verbose=True)

    if dict_config_params['overwrite_cached_classifier']:
        print("\nOverwriting cached classifier.")
        pickle.dump([svc, X_scaler], open( filename_classifier, "wb" ) )

    main_test_images()
    #main_video()


Using already available cached classifier.

Overwriting cached classifier.
[MoviePy] >>>> Building video out_video.mp4
[MoviePy] Writing video out_video.mp4


 97%|█████████▋| 38/39 [00:38<00:01,  1.01s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: out_video.mp4 

